In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os,sys
sys.path.insert(0,"../torchxrayvision/")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import pandas as pd

In [4]:
import torchxrayvision as xrv

In [5]:
dataset = xrv.datasets.COVID19_Dataset(imgpath="../covid-chestxray-dataset/images",
                                      csvpath="../covid-chestxray-dataset/metadata.csv",
                                      views=["PA","AP","AP Supine"])
                                      #views=["PA","AP","AP Supine", "L"])

In [6]:
dataset

{'Aspergillosis': {0.0: 677, 1.0: 2},
 'Aspiration': {0.0: 678, 1.0: 1},
 'Bacterial': {0.0: 628, 1.0: 51},
 'COVID-19': {0.0: 211, 1.0: 468},
 'Chlamydophila': {0.0: 678, 1.0: 1},
 'E.Coli': {0.0: 675, 1.0: 4},
 'Fungal': {0.0: 653, 1.0: 26},
 'H1N1': {0.0: 678, 1.0: 1},
 'Herpes ': {0.0: 676, 1.0: 3},
 'Influenza': {0.0: 675, 1.0: 4},
 'Klebsiella': {0.0: 670, 1.0: 9},
 'Legionella': {0.0: 672, 1.0: 7},
 'Lipoid': {0.0: 671, 1.0: 8},
 'MERS-CoV': {0.0: 669, 1.0: 10},
 'MRSA': {0.0: 678, 1.0: 1},
 'Mycoplasma': {0.0: 674, 1.0: 5},
 'No Finding': {0.0: 662, 1.0: 17},
 'Nocardia': {0.0: 675, 1.0: 4},
 'Pneumocystis': {0.0: 655, 1.0: 24},
 'Pneumonia': {0.0: 28, 1.0: 651},
 'SARS': {0.0: 663, 1.0: 16},
 'Staphylococcus': {0.0: 678, 1.0: 1},
 'Streptococcus': {0.0: 661, 1.0: 18},
 'Tuberculosis': {0.0: 668, 1.0: 11},
 'Varicella': {0.0: 674, 1.0: 5},
 'Viral': {0.0: 173, 1.0: 506}}


COVID19_Dataset num_samples=679 views=['PA', 'AP', 'AP Supine']

In [15]:
dataset.csv = dataset.csv.reset_index()

In [7]:
# csv = pd.read_csv("../covid-chestxray-dataset/metadata.csv")
# csv = csv[csv.view.isin(["PA","AP","AP Supine"])]

In [19]:
len(set(dataset.csv["patientid"]))

412

In [7]:
labels = pd.DataFrame(dict(zip(dataset.pathologies, dataset.labels.T.astype(int))))

In [8]:
labels.columns

Index(['Aspergillosis', 'Aspiration', 'Bacterial', 'COVID-19', 'Chlamydophila',
       'E.Coli', 'Fungal', 'H1N1', 'Herpes ', 'Influenza', 'Klebsiella',
       'Legionella', 'Lipoid', 'MERS-CoV', 'MRSA', 'Mycoplasma', 'No Finding',
       'Nocardia', 'Pneumocystis', 'Pneumonia', 'SARS', 'Staphylococcus',
       'Streptococcus', 'Tuberculosis', 'Varicella', 'Viral'],
      dtype='object')

In [9]:
dataset.csv.groupby("patientid").first().sex.value_counts()

M    230
F    139
Name: sex, dtype: int64

In [13]:
findings = dataset.csv["finding"].str.split("/", expand=True)
findings.columns = ["Finding", "Type", "Genus or Species", "Subtype"]
findings = findings.fillna("Unknown")
findings = findings[findings.Finding == "Pneumonia"]
findings

,Finding,Type,Genus or Species,Subtype
0,Pneumonia,Viral,COVID-19,Unknown
1,Pneumonia,Viral,COVID-19,Unknown
2,Pneumonia,Viral,COVID-19,Unknown
3,Pneumonia,Viral,COVID-19,Unknown
4,Pneumonia,Viral,COVID-19,Unknown
...,...,...,...,...
674,Pneumonia,Unknown,Unknown,Unknown
675,Pneumonia,Unknown,Unknown,Unknown
676,Pneumonia,Unknown,Unknown,Unknown
677,Pneumonia,Unknown,Unknown,Unknown


In [15]:
a = findings.groupby(["Type", "Genus or Species"])[["Subtype"]].count()
a

Subtype
Type       Genus or Species         
Aspiration Unknown                 1
Bacterial  Chlamydophila           1
           E.Coli                  4
           Klebsiella              9
           Legionella              7
           Mycoplasma              5
           Nocardia                4
           Staphylococcus          1
           Streptococcus          18
           Unknown                 2
Fungal     Aspergillosis           2
           Pneumocystis           24
Lipoid     Unknown                 8
Unknown    Unknown                59
Viral      COVID-19              468
           Herpes                  3
           Influenza               4
           MERS-CoV               10
           SARS                   16
           Varicella               5

In [16]:
print(a.to_latex())

\begin{tabular}{llr}
\toprule
      &           &  Subtype \\
Type & Genus or Species &          \\
\midrule
Aspiration & Unknown &        1 \\
Bacterial & Chlamydophila &        1 \\
      & E.Coli &        4 \\
      & Klebsiella &        9 \\
      & Legionella &        7 \\
      & Mycoplasma &        5 \\
      & Nocardia &        4 \\
      & Staphylococcus &        1 \\
      & Streptococcus &       18 \\
      & Unknown &        2 \\
Fungal & Aspergillosis &        2 \\
      & Pneumocystis &       24 \\
Lipoid & Unknown &        8 \\
Unknown & Unknown &       59 \\
Viral & COVID-19 &      468 \\
      & Herpes  &        3 \\
      & Influenza &        4 \\
      & MERS-CoV &       10 \\
      & SARS &       16 \\
      & Varicella &        5 \\
\bottomrule
\end{tabular}



In [15]:
#dmerge.csv[(dmerge.csv.view == "L")]

In [16]:
dmerge.csv["PA/AP"] = (dmerge.csv.view == "PA") | (dmerge.csv.view == "AP")
dmerge.csv["AP Supine"] = dmerge.csv.view == "AP Supine"
dmerge.csv["L"] = dmerge.csv.view == "L"

In [17]:
dmerge.csv["doicite"] = "\cite{" + dmerge.csv.doi.replace("(","").replace(")","") + "}"

In [18]:
df = dmerge.csv.groupby(["doicite"])[["doi","PA/AP", "AP Supine", "L"]].sum()

In [19]:
df.astype(int).head()

,PA/AP,AP Supine,L
doicite,,,
\cite{10.1007/s00068-020-01364-7},3,0,0
\cite{10.1007/s00270-020-02470-0},1,1,0
\cite{10.1007/s00296-020-04584-7},3,0,0
\cite{10.1007/s00701-020-04374-x},0,1,0
\cite{10.1007/s11547-020-01200-3},2,3,0


In [20]:
dmerge.csv.view.unique()

array(['PA', 'AP', 'L', 'AP Supine'], dtype=object)

In [28]:
with pd.option_context('display.max_rows', 10000, 'display.max_columns', 10000):
    display(df.astype(int).sort_values(["PA/AP","AP Supine"], ascending=False))

,PA/AP,AP Supine,L
doicite,,,
\cite{10.6084/m9.figshare.12275009},48,44,0
\cite{10.1148/rg.242035193},11,0,0
\cite{10.1148/radiol.2020201160},9,0,0
\cite{10.1148/radiol.2282030593},5,0,0
\cite{10.1016/j.jfma.2020.02.007},4,0,0
\cite{10.1016/j.jmii.2020.03.008},4,0,0
\cite{10.1016/j.radi.2020.04.002},4,0,0
\cite{10.1056/NEJMoa2001191},4,0,3
\cite{10.1056/nejmc2001272},4,0,0


In [31]:
print(df.astype(int).sort_values(["PA/AP","AP Supine"], ascending=False).to_latex(escape=False))

\begin{tabular}{lrrr}
\toprule
{} &  PA/AP &  AP Supine &  L \\
doicite                                          &        &            &    \\
\midrule
\cite{10.6084/m9.figshare.12275009}              &     48 &         44 &  0 \\
\cite{10.1148/rg.242035193}                      &     11 &          0 &  0 \\
\cite{10.1148/radiol.2020201160}                 &      9 &          0 &  0 \\
\cite{10.1148/radiol.2282030593}                 &      5 &          0 &  0 \\
\cite{10.1016/j.jfma.2020.02.007}                &      4 &          0 &  0 \\
\cite{10.1016/j.jmii.2020.03.008}                &      4 &          0 &  0 \\
\cite{10.1016/j.radi.2020.04.002}                &      4 &          0 &  0 \\
\cite{10.1056/NEJMoa2001191}                     &      4 &          0 &  3 \\
\cite{10.1056/nejmc2001272}                      &      4 &          0 &  0 \\
\cite{10.1148/ryct.2020200034}                   &      4 &          0 &  0 \\
\cite{10.1007/s00068-020-01364-7}                &      3 

In [ ]:
df["PA"]

In [37]:
alreadyhave = pd.read_csv("alreadyhave.txt",header=None)[0]

In [71]:
import doi2bib, doi2bib.crossref

In [179]:
alreadyhave

0            10.1056/nejmc2001272
1            10.1056/NEJMc2001573
2     10.1016/S0140-67362030211-7
3            10.1148/rg.242035193
4     10.1016/S0140-67362030370-6
5       10.1148/radiol.2020200269
6           10.1056/NEJMoa2001191
7         10.1148/ryct.2020200034
8         10.1148/ryct.2020200028
9        10.3346/jkms.2020.35.e79
10      10.1148/radiol.2020200490
11     10.1016/j.jfma.2020.02.007
12              10.1111/all.14238
13     10.1016/j.jmii.2020.03.003
14            10.1093/cid/ciaa199
15          10.3348/kjr.2020.0132
16     10.1016/j.jmii.2020.03.008
17    10.1016/S1473-30992030111-0
18      10.1038/s41591-020-0819-2
19          10.3348/kjr.2020.0112
20         10.1148/cases.20201558
21        10.4103/1817-1737.69106
22         10.1148/cases.20201815
23         10.1148/cases.20201559
24         10.1148/cases.20201394
25      10.1148/radiol.2020201160
26        10.1148/ryct.2020200033
27      10.1148/radiol.2020200274
28      10.1148/radiol.2282030593
29           1

In [138]:
from pybtex.database.input import bibtex
import pybtex.database.output.bibtex

In [178]:
for doi in dmerge.csv.doi[~dmerge.csv.doi.isin(alreadyhave)].dropna().unique():
    try:
        res = doi2bib.crossref.get_bib(doi)
        bib = bibtex.Parser().parse_string(res[1])
        e = bib.entries[bib.entries.keys()[0]]
        e.fields = dict(e.fields)
        del e.fields["url"]
        del e.fields["month"]
        if "volume" in e.fields:
            del e.fields["volume"]
        if "number" in e.fields:
            del e.fields["number"]
        if "pages" in e.fields:
            del e.fields["pages"]
        if "publisher" in e.fields:
            del e.fields["publisher"]
        e.fields["title"] = e.fields["title"].replace("{","").replace("}","")
        e.key = e.fields["doi"].replace("(","").replace(")","")
        bib = bibtex.Parser()
        bib.entries = dict({e.key:e})
        bib.preamble = ""
        a = pybtex.database.output.bibtex.Writer()
        a.write_stream(bib, sys.stdout)
    except:
        pass

@article{10.1016/s0140-67362030211-7,
    author = "Chen, Nanshan and Zhou, Min and Dong, Xuan and Qu, Jieming and Gong, Fengyun and Han, Yang and Qiu, Yang and Wang, Jingli and Liu, Ying and Wei, Yuan and Xia, Jia{\textquotesingle}an and Yu, Ting and Zhang, Xinxin and Zhang, Li",
    doi = "10.1016/s0140-6736(20)30211-7",
    year = "2020",
    title = "Epidemiological and clinical characteristics of 99 cases of 2019 novel coronavirus pneumonia in Wuhan, China: a descriptive study",
    journal = "The Lancet"
}
@article{10.1016/s0140-67362030370-6,
    author = "Silverstein, William Kyle and Stroud, Lynfa and Cleghorn, Graham Edward and Leis, Jerome Allen",
    doi = "10.1016/s0140-6736(20)30370-6",
    year = "2020",
    title = "First imported case of 2019 novel coronavirus in Canada, presenting as mild pneumonia",
    journal = "The Lancet"
}
@article{10.1016/s1473-30992030111-0,
    author = "Cuong, Le Van and Giang, Hoang Thi Nam and Linh, Le Khac and Shah, Jaffer and Sy, Le Van 

@article{10.1016/j.clim.2020.108450,
    author = "Mastaglio, Sara and Ruggeri, Annalisa and Risitano, Antonio M. and Angelillo, Piera and Yancopoulou, Despina and Mastellos, Dimitrios C. and Huber-Lang, Markus and Piemontese, Simona and Assanelli, Andrea and Garlanda, Cecilia and Lambris, John D. and Ciceri, Fabio",
    doi = "10.1016/j.clim.2020.108450",
    year = "2020",
    title = "The first case of COVID-19 treated with the complement C3 inhibitor AMY-101",
    journal = "Clinical Immunology"
}
@article{10.1016/j.crad.2020.04.002,
    author = "Hare, S.S. and Rodrigues, J.C.L. and Nair, A. and Jacob, J. and Upile, S. and Johnstone, A. and Mcstay, R. and Edey, A. and Robinson, G.",
    doi = "10.1016/j.crad.2020.04.002",
    year = "2020",
    title = "The continuing evolution of COVID-19 imaging pathways in the UK: a British Society of Thoracic Imaging expert reference group update",
    journal = "Clinical Radiology"
}
@article{10.1016/j.healun.2020.04.004,
    author = "Aigner

@article{10.7759/cureus.7482,
    author = "Ahmed, Taha and Shah, Ronak J and Rahim, Shab E Gul and Flores, Monica and O{\textquotesingle}Linn, Amy",
    doi = "10.7759/cureus.7482",
    year = "2020",
    title = "Coronavirus Disease 2019 (COVID-19) Complicated by Acute Respiratory Distress Syndrome: An Internist's Perspective",
    journal = "Cureus"
}
@article{10.7759/cureus.7782,
    author = "Siddamreddy, Suman and Thotakura, Ramakrishna and Dandu, Vasuki and Kanuru, Sruthi and Meegada, Sreenath",
    doi = "10.7759/cureus.7782",
    year = "2020",
    title = "Corona Virus Disease 2019 (COVID-19) Presenting as Acute ST Elevation Myocardial Infarction",
    journal = "Cureus"
}
